In [141]:
import os
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import sklearn.decomposition
import mdtraj 
import random
import plotly.graph_objs as go


In [142]:
trajectory = mdtraj.load('/Users/giuliabartocci/Desktop/AA/ensembles/PED00020e001.pdb')


In [150]:
ca_ids = trajectory.topology.select("protein and name CA")
ca_xyz = trajectory.xyz[:,ca_ids]
#starting_matrix = np.zeros((ca_xyz.shape[0], ca_xyz.shape[1]-1,ca_xyz.shape[1]-1))


In [151]:
starting_matrix = np.zeros((ca_xyz.shape[0], ca_xyz.shape[1]-1,ca_xyz.shape[1]-1))
def compute_cos_consecutive(ca_xyz):
    for conformation in range(ca_xyz.shape[0]):
        for i in range(ca_xyz[conformation].shape[0]-1):
            vector_i_i_plus1  =ca_xyz[conformation][i+1]-ca_xyz[conformation][i]
            vector_i_i_plus1/=np.linalg.norm(vector_i_i_plus1)
            for j in range(ca_xyz[conformation].shape[0]-1):
                vector_j_j_plus1=ca_xyz[conformation][j+1]-ca_xyz[conformation][j]
                vector_j_j_plus1/=np.linalg.norm(vector_j_j_plus1)
                cos_i_j=np.dot(vector_i_i_plus1,vector_j_j_plus1)
                starting_matrix[conformation][i][j]=cos_i_j

compute_cos_consecutive(ca_xyz)
mean_cos_tetha=np.mean(starting_matrix,axis=0)
square_diff=(starting_matrix-mean_cos_tetha)**2
variance_cos_tetha=np.mean(square_diff,axis=0)
def site_pair_correlation_parameter(variance_cos_tetha):
    K_ij = [[1 - np.sqrt(2 * variance_cos_tetha[i][j]) for j in range(variance_cos_tetha.shape[1])] for i in range(variance_cos_tetha.shape[0])]
    return np.array(K_ij)
K_ij=site_pair_correlation_parameter(variance_cos_tetha)
o_i=np.mean(K_ij,axis=1)
position = list(range(o_i.shape[0])) 
trace = go.Scatter(x=position, y=o_i, mode='markers')
layout = go.Layout(title='site-specific order parameter',
                   xaxis=dict(title='Position'),
                   yaxis=dict(title='Value',range=[0.14,0.26]),
                   width=1000,  #
                   height=600
                   )
fig = go.Figure(data=[trace], layout=layout)
fig.add_shape(type="line",x0=min(position),y0=0.19,x1=max(position),y1=0.19,line=dict(color="red", width=2, dash="dash"),)
fig.show()

